# Introducción: Proyecto de Machine Learning Parte 1

Al leer un libro de ciencia de datos o seguir un curso, puedes sentir que tienes piezas individuales, pero no sabes cómo unirlas. Dar el siguiente paso y resolver un problema de machine learning completo puede ser desalentador, pero enfrentarse y completar un primer proyecto le dará la confianza para afrontar cualquier problema relacionado con data science. Esta serie de notebooks analizará una solución completa de machine learning con un conjunto de datos del mundo real para que pueda ver cómo se unen todas las piezas.

Seguiremos paso a paso el flujo general de trabajo de machine learning:

1. Limpieza y formateo de datos.
2. Análisis exploratorio de datos
3. Ingeniería y selección de características.
4. Comparar varios modelos de machine learning en una métrica de rendimiento.
5. Ajuste del hiperparámetro en el mejor modelo
6. Evaluar el mejor modelo en el conjunto de pruebas
7. Interpretar los resultados del modelo.
8. Sacar conclusiones y documentar el trabajo.

En el camino veremos cómo cada paso fluye hacia el siguiente y cómo implementar específicamente cada parte en Python.

Definir esta estructura antes de tiempo nos permite ver cómo un paso fluye hacia el otro. Sin embargo, el proceso de machine learning es un procedimiento iterativo, por lo que no siempre seguimos estos pasos de manera lineal. Es posible que volvamos a revisar un paso anterior en función de los resultados de una etapa posterior. Por ejemplo, si bien podemos realizar una selección de características antes de construir cualquier modelo, podemos usar los resultados del modelado para retroceder y seleccionar un conjunto diferente de características. O el modelado puede generar resultados inesperados que signifiquen que queremos explorar nuestros datos desde otro ángulo.

A lo largo de esta serie de notebooks, el objetivo es mostrar cómo todas las diferentes prácticas de data science se unen para formar un proyecto completo. Trato de centrarme más en la implementación de los métodos en lugar de explicarlos en un nivel bajo, pero he proporcionado recursos para aquellos que desean profundizar. Uno de los mejores libros individuales para aprender lo básico e implementar prácticas de machine learning en Python es [Hands-On Machine Learning with Scikit-Learn and Tensorflow](http://shop.oreilly.com/product/0636920052289.do) de [Aurelion Geron](https://twitter.com/aureliengeron?lang=en).

## Definición del problema

El primer paso es comprender el problema que estamos tratando de resolver y los datos disponibles. En este proyecto, trabajaremos con [datos de energía de edificios disponibles públicamente de la ciudad de Nueva York](http://www.nyc.gov/html/gbee/html/plan/ll84_scores.shtml).

**El objetivo es utilizar los datos de energía para construir un modelo que pueda predecir la puntuación Energy Star de un edificio e interpretar los resultados para encontrar los factores que influyen en dicha puntuación**.

Los datos incluyen la puntuación Energy Star, por tanto, se trata de una tarea de machine learning de regresión supervisada:

+ **Supervisado**: tenemos acceso tanto a las características como al objetivo y nuestro objetivo es entrenar un modelo que pueda aprender un mapeo entre los dos.
+ **Regresión**: la puntuación de Energy Star es una variable continua (en este caso va de 0 a 100).

Queremos desarrollar un modelo que sea a la vez **preciso**: puede predecir el puntaje Energy Star cerca del valor real, e **interpretable**: podemos entender las predicciones del modelo. Una vez que conozcamos el objetivo, podemos usarlo para guiar nuestras decisiones a medida que profundizamos en los datos y construimos modelos. Durante el entrenamiento, queremos que el modelo aprenda la relación entre las características y la puntuación, por lo que le damos las características y la respuesta. Luego, para probar cómo de bien ha aprendido el modelo, ¡lo evaluamos en un conjunto de pruebas donde nunca ha visto las respuestas!

## Imports

Usaremos las bibliotecas estándar de ciencia de datos y machine learning: numpy, pandas y scikit-learn. También utilizamos matplotlib y seaborn para visualización.

In [1]:
# Pandas y numpy para manipulación de datos
import pandas as pd
import numpy as np

# Quitamos los warnings sobre el establecimiento de un valor en una copia de un slice
pd.options.mode.chained_assignment = None

# Mostramos hasta 60 columnas de un dataframe
pd.set_option('display.max_columns', 60)

# Visualización Matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# Establece el tamaño de fuente por defecto
plt.rcParams['font.size'] = 24

# Herramienta interna ipython para establecer el tamaño de la figura
from IPython.core.pylabtools import figsize

# Visualizción por Seaborn
import seaborn as sns
sns.set(font_scale = 2)

# División de los datos en entrenamiento y prueba
from sklearn.model_selection import train_test_split

## Limpieza de datos

Contrariamente a lo que pueda parecer, no todos los conjuntos de datos son un grupo de observaciones perfectamente saneado, sin valores perdidos ni anomalías (como puede creerse observando los conjuntos de datos de [mtcars](http://stat.ethz.ch/R-manual/R-devel/library/datasets/html/mtcars.html) e [iris](https://archive.ics.uci.edu/ml/datasets/iris)). Los datos del mundo real son confusos, lo que significa que debemos [limpiarlos y cambiarlos](https://www.springboard.com/blog/data-wrangling/) a un formato aceptable antes de que podamos comenzar el análisis. La limpieza de datos es una parte poco glamurosa pero necesaria de la mayoría de los problemas reales de data science.

Cargaremos nuestros datos en un `dataframe` de pandas, una de las estructuras de datos más útiles de data science. Piense en ello como una hoja de cálculo dentro de Python que podemos manipular, limpiar y visualizar fácilmente. Pandas [tiene muchos métodos](http://pandas.pydata.org/pandas-docs/stable/) para ayudar a que el flujo de información de ciencia de datos / machine learning sea lo más sencillo posible.

In [4]:
# Cargamos los datos en un dataframe 
data = pd.read_csv("./data/Energy_and_Water_Data_Disclosure_for_Local_Law_84_2017__Data_for_Calendar_Year_2016_.csv")

# Display top of dataframe
data.head()

,Order,Property Id,Property Name,Parent Property Id,Parent Property Name,BBL - 10 digits,"NYC Borough, Block and Lot (BBL) self-reported",NYC Building Identification Number (BIN),Address 1 (self-reported),Address 2,Postal Code,Street Number,Street Name,Borough,DOF Gross Floor Area,Primary Property Type - Self Selected,List of All Property Use Types at Property,Largest Property Use Type,Largest Property Use Type - Gross Floor Area (ft²),2nd Largest Property Use Type,2nd Largest Property Use - Gross Floor Area (ft²),3rd Largest Property Use Type,3rd Largest Property Use Type - Gross Floor Area (ft²),Year Built,Number of Buildings - Self-reported,Occupancy,Metered Areas (Energy),Metered Areas (Water),ENERGY STAR Score,Site EUI (kBtu/ft²),Weather Normalized Site EUI (kBtu/ft²),Weather Normalized Site Electricity Intensity (kWh/ft²),Weather Normalized Site Natural Gas Intensity (therms/ft²),Weather Normalized Source EUI (kBtu/ft²),Fuel Oil #1 Use (kBtu),Fuel Oil #2 Use (kBtu),Fuel Oil #4 Use (kBtu),Fuel Oil #5 & 6 Use (kBtu),Diesel #2 Use (kBtu),District Steam Use (kBtu),Natural Gas Use (kBtu),Weather Normalized Site Natural Gas Use (therms),Electricity Use - Grid Purchase (kBtu),Weather Normalized Site Electricity (kWh),Total GHG Emissions (Metric Tons CO2e),Direct GHG Emissions (Metric Tons CO2e),Indirect GHG Emissions (Metric Tons CO2e),Property GFA - Self-Reported (ft²),Water Use (All Water Sources) (kgal),Water Intensity (All Water Sources) (gal/ft²),Source EUI (kBtu/ft²),Release Date,Water Required?,DOF Benchmarking Submission Status,Latitude,Longitude,Community Board,Council District,Census Tract,NTA
0,1,13286,201/205,13286,201/205,1013160001,1013160001,1037549,201/205 East 42nd st.,Not Available,10017,675,3 AVENUE,Manhattan,289356.0,Office,Office,Office,293447,Not Available,Not Available,Not Available,Not Available,1963,2,100,Whole Building,Not Available,Not Available,305.6,303.1,37.8,Not Available,614.2,Not Available,Not Available,Not Available,Not Available,Not Available,5.15506751E7,Not Available,Not Available,38139374.2,1.10827705E7,6962.2,0,6962.2,762051,Not Available,Not Available,619.4,05/01/2017 05:32:03 PM,No,In Compliance,40.750791,-73.973963,6.0,4.0,88.0,Turtle Bay-East Midtown ...
1,2,28400,NYP Columbia (West Campus),28400,NYP Columbia (West Campus),1021380040,1-02138-0040,1084198; 1084387;1084385; 1084386; 1084388; 10...,622 168th Street,Not Available,10032,180,FT WASHINGTON AVENUE,Manhattan,3693539.0,Hospital (General Medical & Surgical),Hospital (General Medical & Surgical),Hospital (General Medical & Surgical),3889181,Not Available,Not Available,Not Available,Not Available,1969,12,100,Whole Building,Whole Building,55,229.8,228.8,24.8,2.4,401.1,Not Available,1.96248472E7,Not Available,Not Available,Not Available,-3.914148026E8,933073441,9330734.4,332365924,9.62613121E7,55870.4,51016.4,4854.1,3889181,Not Available,Not Available,404.3,04/27/2017 11:23:27 AM,No,In Compliance,40.841402,-73.942568,12.0,10.0,251.0,Washington Heights South ...
2,3,4778226,MSCHoNY North,28400,NYP Columbia (West Campus),1021380030,1-02138-0030,1063380,3975 Broadway,Not Available,10032,3975,BROADWAY,Manhattan,152765.0,Hospital (General Medical & Surgical),Hospital (General Medical & Surgical),Hospital (General Medical & Surgical),231342,Not Available,Not Available,Not Available,Not Available,1924,1,100,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,0,0,0,231342,Not Available,Not Available,Not Available,04/27/2017 11:23:27 AM,No,In Compliance,40.840427,-73.940249,12.0,10.0,251.0,Washington Heights South ...
3,4,4778267,Herbert Irving Pavilion & Millstein Hospital,28400,NYP Columbia (West Campus),1021390001,1-02139-0001,1087281; 1076746,161 Fort Washington Ave,177 Fort Washington Ave,10032,161,FT WASHINGTON AVENUE,Manhattan,891040.0,Hos

Echando un vistazo a los datos en bruto, ya podemos ver una serie de problemas que tendremos que resolver. En primer lugar, hay 60 columnas y no sabemos lo que significan muchas de ellas. Todo lo que sabemos de la declaración del problema es que queremos predecir el número en la columna de puntuación. Algunas de las definiciones de otras columnas pueden ser razonablemente adivinadas, pero otras son difíciles de entender (investigo con energía de construcción y aún no pude entender todas las columnas). En machine learning, esto no es realmente un problema, porque dejamos que el modelo decida qué características son importantes. A veces es posible que ni siquiera nos den nombres de columnas ni nos digan lo que estamos pronosticando. No obstante, me gusta entender el problema en la medida de lo posible y, como también queremos interpretar los resultados del modelo, sería una buena idea tener algún conocimiento de las columnas.

Después de mirar el nombre del archivo, `Energy_and_Water_Data_Disclosure_for_Local_Law_84_2017__Data_for_Calendar_Year_2016_.csv`, busqué "Local Law 84." Al hacerlo, se accede a [esta página web](http://www.nyc.gov/html/gbee/html/plan/ll84.shtml) que nos dice que la Ley Local 84 es un requisito de la Ciudad de Nueva York que establece que todos los edificios mayores de $50,000 ft^2$ deben reportar un determinado conjunto de números relacionados con la energía cada año. Después de buscar un poco más, podemos acceder a [este documento pdf](http://www.nyc.gov/html/gbee/downloads/misc/nyc_benchmarking_disclosure_data_definitions_2017.pdf) detalla el significado de cada columna.

Si bien no necesitamos estudiar cada columna, sería una buena idea comprender al menos el objetivo que queremos predecir. Aquí está la definición para el objetivo `score`:

    Una clasificación percentil de 1 a 100 para tipos de edificios específicos, calculada en Portfolio Manager, basada en el consumo de energía autoinformado para el año de informe.
    
Eso parece bastante sencillo: el Energy Star Score es un método para clasificar los edificios en términos de eficiencia energética con 1 para los peores y 100 para los mejores. Es una clasificación de percentil relativo, lo que significa que los edificios se califican entre sí y deben mostrar una distribución uniforme en todo el rango de valores.

## Tipos de datos y valores faltantes

El método `dataframe.info` es una forma rápida de evaluar los datos al mostrar los tipos de datos de cada columna y el número de valores que no faltan. Ya al mirar el marco de datos, puede haber un problema porque los valores faltantes se codifican como "No disponible" en lugar de como np.nan (not a number). Esto significa que las columnas con números no se representarán como numéricas porque los pandas convierten las columnas con cualquier valor de cadena en columnas de todas las cadenas.

In [6]:
# Mostramos los tipos de datos y los valores no faltantes
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11746 entries, 0 to 11745
Data columns (total 60 columns):
Order                                                         11746 non-null int64
Property Id                                                   11746 non-null int64
Property Name                                                 11746 non-null object
Parent Property Id                                            11746 non-null object
Parent Property Name                                          11746 non-null object
BBL - 10 digits                                               11735 non-null object
NYC Borough, Block and Lot (BBL) self-reported                11746 non-null object
NYC Building Identification Number (BIN)                      11746 non-null object
Address 1 (self-reported)                                     11746 non-null object
Address 2                                                     11746 non-null object
Postal Code                                          

Efectivamente, hay una serie de columnas con números que se han registrado como tipos de datos `object`. Estos deberán ser convertidos a tipo de datos `float` antes de que podamos realizar cualquier análisis numérico.

## Convertir datos al tipo correcto

Convertimos las columnas con números en tipos de datos numéricos reemplazando las entradas "Not available" con `np.nan` que se pueden interpretar como `float`. Luego, convertiremos las columnas que contienen valores numéricos (como pies cuadrados o uso de energía) en tipos de datos numéricos.

In [7]:
# Reemplaza todas las ocurrencias de Not Available por numpy not a number
data = data.replace({'Not Available': np.nan})

# Iterar a través de las columnas
for col in list(data.columns):
    # Selecciona las columnas que deben ser numéricas
    if ('ft²' in col or 'kBtu' in col or 'Metric Tons CO2e' in col or 'kWh' in 
        col or 'therms' in col or 'gal' in col or 'Score' in col):
        # Convierte el tipo de dato a float
        data[col] = data[col].astype(float)

In [10]:
data.describe()

,Order,Property Id,DOF Gross Floor Area,Largest Property Use Type - Gross Floor Area (ft²),2nd Largest Property Use - Gross Floor Area (ft²),3rd Largest Property Use Type - Gross Floor Area (ft²),Year Built,Number of Buildings - Self-reported,Occupancy,ENERGY STAR Score,Site EUI (kBtu/ft²),Weather Normalized Site EUI (kBtu/ft²),Weather Normalized Site Electricity Intensity (kWh/ft²),Weather Normalized Site Natural Gas Intensity (therms/ft²),Weather Normalized Source EUI (kBtu/ft²),Fuel Oil #1 Use (kBtu),Fuel Oil #2 Use (kBtu),Fuel Oil #4 Use (kBtu),Fuel Oil #5 & 6 Use (kBtu),Diesel #2 Use (kBtu),District Steam Use (kBtu),Natural Gas Use (kBtu),Weather Normalized Site Natural Gas Use (therms),Electricity Use - Grid Purchase (kBtu),Weather Normalized Site Electricity (kWh),Total GHG Emissions (Metric Tons CO2e),Direct GHG Emissions (Metric Tons CO2e),Indirect GHG Emissions (Metric Tons CO2e),Property GFA - Self-Reported (ft²),Water Use (All Water Sources) (kgal),Water Intensity (All Water Sources) (gal/ft²),Source EUI (kBtu/ft²),Latitude,Longitude,Community Board,Council District,Census Tract
count,11746.000000,1.174600e+04,1.162800e+04,1.174400e+04,3741.000000,1484.000000,11746.000000,11746.000000,11746.000000,9642.000000,11583.000000,10281.000000,10959.000000,9783.000000,10281.000000,9.000000e+00,2.581000e+03,1.321000e+03,5.940000e+02,1.600000e+01,9.360000e+02,1.030400e+04,9.784000e+03,1.150200e+04,1.096000e+04,1.167200e+04,1.166300e+04,1.168100e+04,1.174600e+04,7.762000e+03,7762.000000,11583.000000,9483.000000,9483.000000,9483.000000,9483.000000,9483.000000
mean,7185.759578,3.642958e+06,1.732695e+05,1.605524e+05,22778.682010,12016.825270,1948.738379,1.289971,98.762557,59.854594,280.071484,309.747466,11.072643,1.901441,417.915709,3.395398e+06,3.186882e+06,5.294367e+06,2.429105e+06,1.193594e+06,2.868907e+08,5.048545e+07,5.364578e+05,5.965472e+06,1.768752e+06,4.553657e+03,2.477937e+03,2.076339e+03,1.673739e+05,1.591798e+04,136.172432,385.908029,40.754379,-73.957057,7.140673,15.771275,4977.596647
std,4323.859984,1.049070e+06,3.367055e+05,3.095746e+05,55094.441422,27959.755486,30.576386,4.017484,7.501603,29.993586,8607.178877,9784.731207,127.733868,97.204587,10530.524339,2.213237e+06,5.497154e+06,5.881863e+06,4.442946e+06,3.558178e+06,3.124603e+09,3.914719e+09,4.022606e+07,3.154430e+07,9.389154e+06,2.041639e+05,1.954498e+05,5.931295e+04,3.189238e+05,1.529524e+05,1730.726938,9312.736225,0.080120,0.046337,3.954129,15.674375,13520.422990
min,1.000000,7.365000e+03,5.002800e+04,5.400000e+01,0.000000,0.000000,1600.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.085973e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-4.690797e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-2.313430e+04,0.000000e+00,0.000000e+00,0.000000,0.000000,40.516065,-74.243582,1.000000,1.000000,1.000000
25%,3428.250000,2.747222e+06,6.524000e+04,6.520100e+04,4000.000000,1720.750000,1927.000000,1.000000,100.000000,37.000000,61.800000,65.100000,3.800000,0.100000,103.500000,1.663594e+06,2.550378e+05,2.128213e+06,0.000000e+00,5.698020e+04,4.320254e+06,1.098251e+06,1.176952e+04,1.043673e+06,3.019974e+05,3.287000e+02,1.474500e+02,9.480000e+01,6.699400e+04,2.595400e+03,27.150000,99.400000,40.707226,-73.984662,4.000000,4.000000,100.000000
50%,6986.500000,3.236404e+06,9.313850e+04,9.132400e+04,8654.000000,5000.000000,1941.000000,1.000000,100.000000,65.000000,78.500000,82.500000,5.300000,0.500000,129.400000,4.328815e+06,1.380138e+06,4.312984e+06,0.000000e+00,2.070020e+05,9.931240e+06,4.103962e+06,4.445525e+04,1.855196e+06,5.416312e+05,5.002500e+02,2.726000e+02,1.718000e+02,9.408000e+04,4.692500e+03,45.095000,124.900000,40.759130,-73.962810,7.000000,9.000000,201.000000
75%,11054.500000,4.409092e+06,1.596140e+05,1.532550e+05,20000.000000,12000.000000,1966.000000,1.000000,100.000000,85.000000,97.600000,102.500000,9.200000,0.700000,167.200000,4.938947e+06,4.445808e+06,6.514520e+06,4.293825e+06,2.918332e+0